In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import numpy as np
import pandas as pd
import gc

In [3]:
path = "../../Zillow4/Data/"
print("Read the properties and merge with coord")
coord = pd.read_csv(path + 'renamed_properties_2017.csv', usecols = ["id_parcel", "latitude", "longitude"])
print "Size of the properties data frame: ", coord.shape

Read the properties and merge with coord
Size of the properties data frame:  (2985217, 3)


In [4]:
index = pd.isnull(coord["latitude"])
coord = coord[~index]
print coord.shape

(2982285, 3)


In [5]:
coord['latitude'] = coord['latitude'] / 1e6
coord['longitude'] = coord['longitude'] / 1e6

In [6]:
print("Read train file to obtain some statistic about the features and target")
train = pd.read_csv(path + 'train_2017.csv', parse_dates=['transactiondate'])
train.columns = ["id_parcel", "logerror", "date"]

Read train file to obtain some statistic about the features and target


In [7]:
train = pd.merge(train, coord, on="id_parcel", how="left")
print train.shape

(77613, 5)


In [8]:
index = pd.isnull(train["latitude"])
train = train[~index]
print train.shape

(77579, 5)


In [9]:
X = train[['latitude','longitude']].as_matrix()

In [10]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [19]:
col = ["n_clusters", "score"]
clusters = pd.DataFrame(columns=col)
n_clusters = [10*i for i in range(1,40)]

for nn in n_clusters:
    kmeans = MiniBatchKMeans(n_clusters=nn, batch_size=1000)
    kmeans_labels = kmeans.fit_predict(X)
    
    score = silhouette_score(X, kmeans_labels, sample_size = 20000, random_state = 0)
    
    clusters = clusters.append(pd.DataFrame([[nn, score]], columns=col))
    print nn, score
                               

10 0.424699824674
20 0.37539814259
30 0.380108854809
40 0.380914325188
50 0.374500526069
60 0.37685319203
70 0.372733593747
80 0.396824172547
90 0.384353558147
100 0.36455296284
110 0.380547026898
120 0.377966019054
130 0.382788973157
140 0.376613635603
150 0.381481305889
160 0.382809438493
170 0.374814328069
180 0.379693157466
190 0.37083823402
200 0.384572570436
210 0.36731140615
220 0.376882597255
230 0.378794353102
240 0.363449979414
250 0.36757963141
260 0.375108230991
270 0.367720950231
280 0.361656952659
290 0.367067340402
300 0.374392745275
310 0.367819885649
320 0.372335573762
330 0.362189530564
340 0.370704412741
350 0.378831404881
360 0.360517750962
370 0.360554326092
380 0.373062201016
390 0.362099614168


In [20]:
clusters = clusters.sort_values("score")
#clusters.to_csv("n_clusters.csv")
print clusters

   n_clusters     score
0       360.0  0.360518
0       370.0  0.360554
0       280.0  0.361657
0       390.0  0.362100
0       330.0  0.362190
0       240.0  0.363450
0       100.0  0.364553
0       290.0  0.367067
0       210.0  0.367311
0       250.0  0.367580
0       270.0  0.367721
0       310.0  0.367820
0       340.0  0.370704
0       190.0  0.370838
0       320.0  0.372336
0        70.0  0.372734
0       380.0  0.373062
0       300.0  0.374393
0        50.0  0.374501
0       170.0  0.374814
0       260.0  0.375108
0        20.0  0.375398
0       140.0  0.376614
0        60.0  0.376853
0       220.0  0.376883
0       120.0  0.377966
0       230.0  0.378794
0       350.0  0.378831
0       180.0  0.379693
0        30.0  0.380109
0       110.0  0.380547
0        40.0  0.380914
0       150.0  0.381481
0       130.0  0.382789
0       160.0  0.382809
0        90.0  0.384354
0       200.0  0.384573
0        80.0  0.396824
0        10.0  0.424700


In [11]:
X = train[['latitude','longitude']].as_matrix()
X2 = coord[['latitude','longitude']].as_matrix()

In [15]:
n_clusters = [40, 80, 130, 160, 200, 350, 250, 300, 400, 450, 500, 550]

for ii, nn in enumerate(n_clusters):
    kmeans = MiniBatchKMeans(n_clusters=nn, batch_size=1000)
    kmeans.fit(X)
    
    name = "cluster" + str(ii)
    coord[name] = kmeans.predict(X2) 

In [16]:
coord.head()

,id_parcel,latitude,longitude,cluster0,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7,cluster8,cluster9,cluster10,cluster11
0,10754147,34.144442,-118.654084,1,37,24,78,165,206,135,205,157,360,336,26
1,10759547,34.140430,-118.625364,1,42,101,34,165,206,135,205,157,291,437,525
2,10843547,33.989359,-118.394633,19,56,71,136,84,293,167,166,329,34,199,369
3,10859147,34.148863,-118.437206,10,70,53,134,78,165,85,220,112,280,23,350
4,10879947,34.194168,-118.385816,31,17,46,22,172,1,60,294,311,411,134,190


In [17]:
coord.drop(['latitude','longitude'], axis=1).to_csv("../Data/clusters_2017_v3.csv", index=False)